In [1]:
import sys
import numpy as np
import pandas as pd
from pathlib import Path


sys.path.append("../src/damagescanner")

data_path = Path("..")

import vector
from raster import RasterScanner
from vector import VectorScanner

In [2]:
class DamageScanner(object):
    """DamageScanner - a directe damage assessment toolkit"""

    def __init__(self, data_path, exposure_data, hazard_data, curves=None, maxdam=None):
        """Prepare the input for a damage assessment"""

        # specify the basics
        self.data_path = data_path
        self.osm = False
        self.default_curves = False
        self.default_maxdam = False

        # Specify paths to exposure data
        self.exposure_path = Path(data_path / "exposure" / exposure_data)

        if self.exposure_path.suffix in [".tif", ".tiff", ".nc"]:
            self.assessment_type = "raster"

        elif self.exposure_path.suffix in [
            ".shp",
            ".gpkg",
            ".pbf",
            ".geofeather",
            ".geoparquet",
        ]:
            self.assessment_type = "vector"

            if self.exposure_path.suffix == ".pbf":
                self.osm = True
        else:
            raise ImportError(
                ": The exposure data should be a a shapefile, geopackage, geoparquet, osm.pbf, geotiff or netcdf file."
            )

        # Specify path to hazard data
        if not isinstance(hazard_data, list):
            self.hazard_path = Path(data_path / "hazard" / hazard_data)
        else:
            self.hazard_path = [Path(data_path / "hazard" / x) for x in hazard_data]

        # Collect vulnerability curves
        if curves is None:
            self.curves = "https://zenodo.org/records/10203846/files/Table_D2_Multi-Hazard_Fragility_and_Vulnerability_Curves_V1.0.0.xlsx?download=1"

            raise ImportWarning(
                "You have decided to choose the default set of curves. Make sure the exposure data aligns with these curves: https://zenodo.org/records/10203846"
            )
            self.default_curves = True

        elif isinstance(curves, pd.DataFrame):
            self.curves = curves
        else:
            self.curves = Path(data_path / "vulnerability" / curves)

        # Collect maxdam information
        if maxdam is None:
            self.maxdam = "https://zenodo.org/records/10203846/files/Table_D3_Costs_V1.0.0.xlsx?download=1"

            raise ImportWarning(
                "You have decided to choose the default set of maximum damages. Make sure the exposure data aligns with these maximum damages: https://zenodo.org/records/10203846"
            )
            self.default_maxdam = True

        elif isinstance(maxdam, dict):
            self.maxdam = maxdam
        else:
            self.maxdam = Path(data_path / "vulnerability" / maxdam)

    # def exposure(self):

    # def vulnerability(self):

    def calculate(self, hazard_type=None, save_output=False, **kwargs):
        """Damage assessment. Can be a specific hazard event, or a specific single hazard footprint, or a list of events/footprints."""

        if not hasattr(self, "assessment_type"):
            raise ImportError("Please run .prepare() first to set up the assessment")

        if self.assessment_type == "raster":
            return RasterScanner(
                exposure_file=self.exposure_path,
                hazard_file=self.hazard_path,
                curve_path=self.curves,
                maxdam_path=self.maxdam,
                save=save_output,
            )

        elif self.assessment_type == "vector":
            if self.default_curves:
                if hazard_type == "flood":
                    sheet_name = "F_Vuln_Depth"
                elif hazard_type == "windstorm":
                    sheet_name = "W_Vuln_V10m"

            # specificy essential data input characteristics
            if "cell_size" in kwargs:
                self.cell_size = kwargs.get("cell_size")  # 0.01666,
            else:
                self.cell_size = 5

            if "exp_crs" in kwargs:
                self.exp_crs = kwargs.get("exp_crs")
            else:
                self.exp_crs = 4326

            if "haz_crs" in kwargs:
                self.haz_crs = kwargs.get("haz_crs")
            else:
                self.haz_crs = 4326

            if "object_col" in kwargs:
                self.object_col = kwargs.get("object_col")
            else:
                self.object_col = "landuse"

            if "hazard_col" in kwargs:
                self.hazard_col = kwargs.get("hazard_col")
            else:
                self.hazard_col = "inun_val"

            if "lat_col" in kwargs:
                self.lat_col = kwargs.get("lat_col")
            else:
                self.lat_col = "y"

            if "lon_col" in kwargs:
                self.lon_col = kwargs.get("lon_col")
            else:
                self.lon_col = "x"

            if "centimers" in kwargs:
                self.centimers = kwargs.get("centimers")  # 0.01666,#5,
            else:
                self.centimers = False

            if self.osm:
                if "buildings" in kwargs:
                    self.exposure_data = vector.buildings(self)
                    self.exposure_data = self.exposure_data.rename(
                        {"building": "element_type"}, axis=1
                    )

                elif "roads" in kwargs:
                    self.exposure_data = vector.cis(self, infra_type="road")
                    self.exposure_data = self.exposure_data.rename(
                        {"highway": "element_type"}, axis=1
                    )

                elif "landuse" in kwargs:
                    self.exposure_data = vector.landuse(self)
                    self.exposure_data = self.exposure_data.rename(
                        {"landuse": "element_type"}, axis=1
                    )
                else:
                    raise RuntimeError(
                        "When using OSM data, you need to specify the object type (e.g. road, buildings, landuse)"
                    )
            else:
                self.exposure_data = self.exposure_path

            return VectorScanner(
                exposure_file=self.exposure_data,
                hazard_file=self.hazard_path,
                curve_path=self.curves,
                maxdam_path=self.maxdam,
                cell_size=self.cell_size,  # 0.01666,#5,
                exp_crs=self.exp_crs,  # 28992,
                haz_crs=self.haz_crs,  # 4326,
                object_col=self.object_col,  #'landuse',
                hazard_col=self.hazard_col,
                lat_col=self.lat_col,
                lon_col=self.lon_col,
                centimeters=self.centimers,  # False,
                save=save_output,
            )

    def risk(self):
        """Risk assessment"""
        pass

In [3]:
kampen = DamageScanner(
    data_path=data_path / "data" / "kampen",
    exposure_data="landuse_map.tif",
    hazard_data="windstorm.nc",
    curves="curves.csv",
    maxdam="maxdam.csv",
)

output = kampen.calculate()

In [17]:
maxdam = {
    "grass": 5,
    "forest": 10,
    "orchard": 50,
    "residential": 200,
    "education": 200,
    "industrial": 300,
    "retail": 300,
    "railway": 100,
    "farmland": 10,
    "cemetery": 15,
    "construction": 10,
    "meadow": 5,
    "farmyard": 5,
    "scrub": 5,
    "allotments": 10,
    "reservoir": 5,
    "static_caravan": 100,
    "commercial": 300,
    "unclassified": 100,
    "primary": 100,
    "secondary": 100,
    "tertiary": 100,
    "residential": 100,
    "trunk": 100,
    "trunk_link": 100,
    "motorway": 100,
    "motorway_link": 100,
    "primary_link": 100,
    "secondary_link": 100,
    "tertiary_link": 100,
    "yes": 300,
    "house": 300,
    "hospital": 300,
    "hotel": 300,
    "apartments": 300,
    "roof": 300,
    "houseboat": 300,
    "greenhouse": 300,
    "school": 300,
    "church": 300,
    "civic": 300,
    "shed": 300,
    "garage": 300,
    "office": 300,
    "tech_cab": 300,
    "fire_station": 300,
    "public": 300,
    "service": 300,
    "factory": 300,
    "farm_auxiliary": 300,
    "storage_tank": 300,
    "parking": 300,
    "bicycle shed": 300,
}

curves = np.array([[0, 0], [0.5, 0.2], [1, 0.4], [1.5, 0.6], [2.0, 0.8], [2.5, 1]])

curves = np.concatenate(
    (curves, np.transpose(np.array([curves[:, 1]] * (len(maxdam) - 1)))), axis=1
)

curves = pd.DataFrame(curves)
curves.columns = ["depth"] + list(maxdam.keys())
curves.set_index("depth", inplace=True)

In [18]:
maxdam_df = pd.DataFrame.from_dict(maxdam, orient="index").reset_index()

In [19]:
maxdam_df.columns = ["landuse", "damage"]

In [20]:
maxdam_df.to_csv(
    "C:\\projects\\DamageScanner\\data\\kampen\\vulnerability\\maxdam_osm.csv",
    index=False,
)

In [21]:
curves.to_csv(
    "C:\\projects\\DamageScanner\\data\\kampen\\vulnerability\\curves_osm.csv"
)

In [12]:
kampen = DamageScanner(
    data_path=data_path / "data" / "jamaica",
    exposure_data="jamaica-latest.osm.pbf",
    hazard_data="FD_1in1000.tif",
    curves=curves,
    maxdam=maxdam,
)

In [13]:
kampen.exposure_data.element_type.unique()

AttributeError: 'DamageScanner' object has no attribute 'exposure_data'

In [11]:
kampen.calculate(buildings=True, cell_size=0.000833333, exp_crs=4326, haz_crs=4326)

extract multipolygons: 100%|████████████████████████████████████████████████| 998092/998092 [00:38<00:00, 25732.43it/s]


PROGRESS: Exposure and hazard data loaded


  2%|█▌                                                                         | 1331/61678 [00:00<00:15, 3912.44it/s]


KeyError: 'supermarket'

In [13]:
kampen = DamageScanner(
    data_path=data_path / "data" / "kampen",
    exposure_data="jamaica-latest.osm.pbf",
    hazard_data="windstorm.nc",
    curves=curves,
    maxdam=maxdam,
)

In [ ]:
%%time
kampen.calculate(
    hazard_type="windstorm",
    exp_crs=28992,
    haz_crs=4326,
    cell_size=0.166667,
    hazard_col="FX",
    lat_col="Latitude",
    lon_col="Longitude",
    centimeters=True,
)